<a href="https://colab.research.google.com/github/vinismachadoo/web-scrapping-farm-rio/blob/main/catalogo_farm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests, threading
from multiprocessing.pool import ThreadPool, Pool
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url_base = 'https://www.farmrio.com.br'
subcategorias_farm = []

print('------ Procurando links no site da FARM ------')
pagina_inicial = requests.get(url_base)
soup = BeautifulSoup(pagina_inicial.content, 'html.parser')
menu = soup.find_all('a', {'class':'menu__link menu__link--level-3'})
for m in menu:
    if 'http' not in m.get('href'):
        subcategorias_farm.append(url_base+m.get('href'))
    else:
        subcategorias_farm.append(m.get('href'))

print(f'-- Encontradas {len(subcategorias_farm)} sublinks')

------ Procurando links no site da FARM ------
-- Encontradas 79 sublinks


In [ ]:
def encontra_url_categorias(subcategoria):
    pagina_categoria = requests.get(subcategoria)
    soup = BeautifulSoup(pagina_categoria.text, 'html.parser')
    # encontra todas as tag script do html
    scripts = soup.find_all('script')
    for script in scripts:
        s = script.string
        # se o script contiver 'var pagecount' pega o link paginado
        if s is not None and 'var pagecount' in s:
            cat_url = s.split("load('")[1].split("' + page")[0]
            lista_url_cartegorias.append(url_base+cat_url)
        else:
            pass

In [ ]:
lista_url_cartegorias = []
print('------ Acessando categorias da FARM ------')
# executa a função com 15 fios em paralelo para ser mais rápido
ThreadPool(15).map(encontra_url_categorias, subcategorias_farm)
print('-- Todas os links visitados')
# deleta os url duplicados
urls_farm = list(dict.fromkeys(lista_url_cartegorias))
print(f'-- Encontradas {len(urls_farm)} subcategorias')

------ Acessando categorias da FARM ------
-- Todas os links visitados
-- Encontradas 35 subcategorias


In [ ]:
def encontra_skus_farm(url_farm):
    page_number = 1
    while True:
        page = requests.get(f'{url_farm}{page_number}')
        # se o url for vazio, interrompe o laço
        if page.text == '':
            break
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            # encontra todas as tags div da classe identificada
            skus = soup.find_all("div", {"class":"shelf__product shelf-product js-vitrine-interativa"})
            for s in skus:
                # adiciona na lista o atributo do div
                lista_skus.append(s.get('data-product-id'))
        page_number += 1

In [ ]:
lista_skus = []
print('------ Pegando SKUS da FARM ------')
ThreadPool(20).map(encontra_skus_farm, urls_farm)
print('-- Site da FARM totalmente carregado')
skus_farm = list(dict.fromkeys(lista_skus))
print(f'-- Encontrados {len(skus_farm)} SKUS')

------ Pegando SKUS da FARM ------
-- Site da FARM totalmente carregado
-- Encontrados 1614 SKUS


In [ ]:
def raspador_farm(sku):
    try:
        r = requests.get(f'https://www.farmrio.com.br/api/catalog_system/pub/products/search/?fq=productId:{sku}')
        product_data = r.json()[0]
    except:
        pass
    else:
        try:
            id = product_data['productId']
        except:
            id = ''
        try:
            nome = product_data['productName']
        except:
            nome = ''
        try:
            url = product_data['link']
        except:
            url = ''
        try:
            marca = product_data['Marca'][0]
        except:
            marca = ''
        try:
            colecao = product_data['Coleção'][0]
        except:
            colecao = ''
        try:
            composicao = product_data['Composição'][0]
        except:
            composicao = ''
        try:
            estampa = product_data['Nome Estampa'][0]
        except:
            estampa = ''
        # dicionário com as informações geral do produto
        general_product_info = {
        'ID': id,
        'Nome': nome,
        'URL': url,
        'Marca': marca,
        'Colecao': colecao,
        'Composicao': composicao,
        'Estampa': estampa,
        }

        for item in product_data['items']:
            try:
                tamanho = item['Tamanho'][0]
            except:
                tamanho = ''
            try:
                ean = item['ean']
            except:
                ean = ''
            try:
                em_falta = item['sellers'][0]['commertialOffer']['GetInfoErrorMessage']
            except:
                em_falta = ''
            try:
                preco_default = item['sellers'][0]['commertialOffer']['ListPrice']
            except:
                preco_default = ''
            try:
                preco_atual = item['sellers'][0]['commertialOffer']['Price']
            except:
                preco_atual = ''
            # dicionário para cada tamanho do produto
            size_product_info = {
                'Tamanho': tamanho,
                'EAN': ean,
                'Em falta': em_falta,
                'Preço default': preco_default,
                'Preco atual': preco_atual
            }
            # adiciona na lista cada tamanho junto com suas especificações gerais
            farm.append({**general_product_info, **size_product_info})

In [ ]:
farm = []
print('------ Montando catálogo FARM ------')
ThreadPool(100).map(raspador_farm, skus_farm)
print(f'-- Catálogo completo com {len(farm)} EANs')

------ Montando catálogo FARM ------
-- Catálogo completo com 7253 EANs


In [ ]:
df_farm = pd.DataFrame(farm)
df_farm

,ID,Nome,URL,Marca,Colecao,Composicao,Estampa,Tamanho,EAN,Em falta,Preço default,Preco atual
0,46185,Choker Multicolorida,https://www.farmrio.com.br/choker-multicolorid...,Farm,Provisorio,Indefinida,,U,2881682276U,Code: withoutPriceFulfillment Status:error Mes...,0.0,0.0
1,46180,Colar Contrapinado Tons Frios,https://www.farmrio.com.br/colar-contrapinado-...,Farm,Provisorio,Indefinida,,U,2881642276U,None,198.0,198.0
2,566,Bolsa Abacaxi Couro Grande,https://www.farmrio.com.br/bolsa-abacaxi-couro...,Farm,Provisorio,100% Couro,,U,2449510026U,None,698.0,698.0
3,48357,Regata Danca Dos Tucanos,https://www.farmrio.com.br/regata-danca-dos-tu...,Farm,Alto Verao 2021,100% Viscose,DANCA DOS TUCANOS_PRETO,PP,29072810805PP,None,149.0,149.0
4,48357,Regata Danca Dos Tucanos,https://www.farmrio.com.br/regata-danca-dos-tu...,Farm,Alto Verao 2021,100% Viscose,DANCA DOS TUCANOS_PRETO,P,29072810805P,None,149.0,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7248,38290,Top Mix Rendas,https://www.farmrio.com.br/top-mix-rendas-off-...,Farm,Provisorio,100% Algodão,,G,2778620024G,None,298.0,208.0
7249,36676,Regata Desejo Sorte,https://www.farmrio.com.br/regata-desejo-sorte...,Farm,Provisorio,100% Viscose,,PP,2785280024PP,Code: withoutStock Status:error Message: O ite...,0.0,0.0
7250,36676,Regata Desejo Sorte,https://www.farmrio.com.br/regata-desejo-sorte...,Farm,Provisorio,100% Viscose,,P,2785280024P,None,89.0,89.0
7251,36676,Regata Desejo Sorte,https://www.farmrio.com.br/regata-desejo-sorte...,Farm,Provisorio,100% Viscose,,M,2785280024M,None,89.0,89.0
